In [2]:
import os
import sgml, dproc, sgml, sgpp, sgutil, cupp, sgnn
import pandas as pd
import polars as pl
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.pipeline import make_pipeline
import warnings 
warnings.simplefilter('ignore', FutureWarning)

2025-05-02 08:07:07.413334: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1746173227.424936   21786 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1746173227.428526   21786 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1746173227.437614   21786 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1746173227.437627   21786 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1746173227.437629   21786 computation_placer.cc:177] computation placer alr

In [3]:
df_train = pd.read_pickle('data/train_proc.pkl')
df_test = pd.read_pickle('data/test_proc.pkl')
sc = sgutil.SGCache('img', 'result', 'model')

In [4]:
import lightgbm as lgb
import xgboost as xgb
import catboost as cb
from sklearn.metrics import root_mean_squared_error
from sklearn.model_selection import KFold, cross_validate, cross_val_score, ShuffleSplit, train_test_split
kf = KFold(3, shuffle = True, random_state = 123)
ss = ShuffleSplit(1, train_size = 0.6, random_state = 123)

target = 'price'
X_all = df_test.columns.tolist()
def get_validation_splitter(validation_fraction):
    return lambda x: train_test_split(x, test_size = validation_fraction)

config = {
    'predict_func': lambda m, df, X: pd.Series(m.predict(df[X]), index = df.index).rename(target),
    'score_func': lambda df, prds: root_mean_squared_error(df[target], prds),
    'validation_splitter': get_validation_splitter,
    'progress_callback': sgml.ProgressCallBack(), 
    'return_train_scores': True,
    'y': target,
}

lgb_adapter = sgml.LGBMAdapter(lgb.LGBMRegressor, progress = 50)
xgb_adapter = sgml.XGBAdapter(xgb.XGBRegressor, progress = 50)
cb_adapter = sgml.CBAdapter(cb.CatBoostRegressor, progress = 50)

- 전처리 과정에서 정리에 효과가 있었던 변수, brand2, model2를 brand와 model을 대신해서 투입
- transmission을 분해한 speed와 method 분리한 변수로 투입 
- engine을 제외하고, HP, displacement, engine_type,fuel_type를 투입합니다.
- 이상점과, 공선성이 많은 데이터 이므로, colsample_bytree와 subsample의 효과성을 살펴볼 만 합니다.

# LGB1

In [42]:
hparams = {
    'model_params': {
        'n_estimators': 100, 'learning_rate': 0.05, 'colsample_bytree': 0.25, 'subsample': 0.5, 'metric': 'RMSE',
        'subsample_freq': 1
    }, 
    'X_num' : ['model_year', 'milage', 'clean_title', 'HP', 'displacement', 'speed'],
    'X_cat' : ['brand2', 'model2', 'fuel_type', 'ext_col', 'int_col', 'accident', 'engine_type', 'fuel_type_engine', 'method'],
    'cat': {'handle_unknown': 'use_encoded_value', 'unknown_value': -1, 'encoded_missing_value': -1},
    'validation_fraction': 0.0
}
result_lgb = sc.cv_result('lgb1', df_train, kf, hparams, config, lgb_adapter, rerun = 0)
np.mean(result_lgb['valid_scores']), result_lgb['valid_scores']

(np.float64(73126.03774675484),
 [np.float64(72654.94299535119),
  np.float64(73774.6678137179),
  np.float64(72948.50243119543)])

# LGB2

In [53]:
hparams = {
    'model_params': {
        'num_leaves': 15, 'n_estimators': 100, 'learning_rate': 0.05, 'colsample_bytree': 0.25, 'subsample': 0.5, 'metric': 'RMSE',
        'subsample_freq': 1
    }, 
    'X_num' : ['model_year', 'milage', 'clean_title', 'HP', 'displacement', 'speed'],
    'X_cat' : ['brand2', 'model2', 'fuel_type', 'ext_col', 'int_col', 'accident', 'engine_type', 'fuel_type_engine', 'method'],
    'cat': {'handle_unknown': 'use_encoded_value', 'unknown_value': -1, 'encoded_missing_value': -1},
    'validation_fraction': 0.0
}
result_lgb = sc.cv_result('lgb2', df_train, kf, hparams, config, lgb_adapter, rerun = 0)
np.mean(result_lgb['valid_scores']), result_lgb['valid_scores']

Fold:   0%|          | 0/3 [00:00<?, ?it/s]

Round:   0%|          | 0/100 [00:00<?, ?it/s]

Round:   0%|          | 0/100 [00:00<?, ?it/s]

Round:   0%|          | 0/100 [00:00<?, ?it/s]

(np.float64(73103.69088746003),
 [np.float64(72631.17496246735),
  np.float64(73764.29274176608),
  np.float64(72915.60495814668)])

# CB1

In [110]:
hparams = {
    'model_params': {'n_estimators': 350, 'learning_rate': 0.03, 'colsample_bylevel': 0.5, 'subsample': 0.5}, 
    'X_num' : ['model_year', 'milage', 'clean_title', 'HP', 'displacement', 'speed'],
    'X_cat' : ['brand2', 'model2', 'fuel_type', 'method', 'ext_col', 'int_col', 'accident', 'engine_type', 'fuel_type_engine'],
    'cat': {'handle_unknown': 'use_encoded_value', 'unknown_value': -1, 'encoded_missing_value': -1},
    'validation_fraction': 0.0
}
result_cb = sc.cv_result('cb1', df_train, kf, hparams, config, cb_adapter, rerun = 0)
np.mean(result_cb['valid_scores']), result_cb['valid_scores']

Fold:   0%|          | 0/3 [00:00<?, ?it/s]

Round:   0%|          | 0/350 [00:00<?, ?it/s]

Round:   0%|          | 0/350 [00:00<?, ?it/s]

Round:   0%|          | 0/350 [00:00<?, ?it/s]

(np.float64(72826.97552013601),
 [np.float64(72369.80797852752),
  np.float64(73450.82948886989),
  np.float64(72660.28909301062)])

# CB2

In [127]:
hparams = {
    'model_params': {'max_depth': 5, 'n_estimators': 850, 'learning_rate': 0.03, 'colsample_bylevel': 0.5, 'subsample': 0.5}, 
    'X_num' : ['model_year', 'milage', 'clean_title', 'HP', 'displacement', 'speed'],
    'X_cat' : ['brand2', 'model2', 'fuel_type', 'method', 'ext_col', 'int_col', 'accident', 'engine_type', 'fuel_type_engine'],
    'cat': {'handle_unknown': 'use_encoded_value', 'unknown_value': -1, 'encoded_missing_value': -1},
    'validation_fraction': 0.0
}
result_cb = sc.cv_result('cb2', df_train, kf, hparams, config, cb_adapter, rerun = 1)
np.mean(result_cb['valid_scores']), result_cb['valid_scores']

Fold:   0%|          | 0/3 [00:00<?, ?it/s]

Round:   0%|          | 0/850 [00:00<?, ?it/s]

Round:   0%|          | 0/850 [00:00<?, ?it/s]

Round:   0%|          | 0/850 [00:00<?, ?it/s]

(np.float64(72790.98690276669),
 [np.float64(72319.41451040065),
  np.float64(73455.37773399118),
  np.float64(72598.16846390825)])

# XGB1

In [155]:
hparams = {
    'model_params': {
        'max_depth': 3, 'n_estimators': 550, 'learning_rate': 0.01, 'colsample_bytree': 0.25, 'subsample': 0.5,
        'enable_categorical': True
    }, 
    'X_num' : ['model_year', 'milage', 'clean_title', 'HP', 'displacement', 'speed'],
    'X_cat' : ['brand2', 'model2', 'fuel_type', 'ext_col', 'int_col', 'accident', 'engine_type', 'fuel_type_engine', 'method'],
    'cat': {'handle_unknown': 'use_encoded_value', 'unknown_value': -1, 'encoded_missing_value': -1},
    'validation_fraction': 0.0
}
result_xgb = sc.cv_result('xgb1', df_train, kf, hparams, config, xgb_adapter, rerun = 0)
np.mean(result_xgb['valid_scores']), result_xgb['valid_scores']

Fold:   0%|          | 0/3 [00:00<?, ?it/s]

Round:   0%|          | 0/550 [00:00<?, ?it/s]

Round:   0%|          | 0/550 [00:00<?, ?it/s]

Round:   0%|          | 0/550 [00:00<?, ?it/s]

(np.float64(73102.25545583667),
 [np.float64(72782.4386748881),
  np.float64(73746.93383663215),
  np.float64(72777.39385598978)])

# XGB2

In [164]:
hparams = {
    'model_params': {
        'max_depth': 2, 'n_estimators': 550, 'learning_rate': 0.01, 'colsample_bytree': 0.25, 'subsample': 0.5,
        'enable_categorical': True
    }, 
    'X_num' : ['model_year', 'milage', 'clean_title', 'HP', 'displacement', 'speed'],
    'X_cat' : ['brand2', 'model2', 'fuel_type', 'ext_col', 'int_col', 'accident', 'engine_type', 'fuel_type_engine', 'method'],
    'cat': {'handle_unknown': 'use_encoded_value', 'unknown_value': -1, 'encoded_missing_value': -1},
    'validation_fraction': 0.0
}
result_xgb = sc.cv_result('xgb2', df_train, kf, hparams, config, xgb_adapter, rerun = 0)
np.mean(result_xgb['valid_scores']), result_xgb['valid_scores']

Fold:   0%|          | 0/3 [00:00<?, ?it/s]

Round:   0%|          | 0/550 [00:00<?, ?it/s]

Round:   0%|          | 0/550 [00:00<?, ?it/s]

Round:   0%|          | 0/550 [00:00<?, ?it/s]

(np.float64(73180.96601080298),
 [np.float64(72857.16472818457),
  np.float64(73831.12021971532),
  np.float64(72854.61308450907)])

In [162]:
result_xgb['model_result'][0]['valid_result'].idxmin()

metric  set         
rmse    validation_0    549
        validation_1    548
dtype: int64

# CB3

In [180]:
hparams = {
    'model_params': {'max_depth': 4, 'n_estimators': 900, 'learning_rate': 0.03, 'colsample_bylevel': 0.5, 'subsample': 0.5}, 
    'X_num' : ['model_year', 'milage', 'clean_title', 'HP', 'displacement', 'speed'],
    'X_cat' : ['brand2', 'model2', 'fuel_type', 'method', 'ext_col', 'int_col', 'accident', 'engine_type', 'fuel_type_engine'],
    'cat': {'handle_unknown': 'use_encoded_value', 'unknown_value': -1, 'encoded_missing_value': -1},
    'validation_fraction': 0.0
}
result_cb = sc.cv_result('cb3', df_train, kf, hparams, config, cb_adapter, rerun = 0)
np.mean(result_cb['valid_scores']), result_cb['valid_scores']

Fold:   0%|          | 0/3 [00:00<?, ?it/s]

Round:   0%|          | 0/900 [00:00<?, ?it/s]

Round:   0%|          | 0/900 [00:00<?, ?it/s]

Round:   0%|          | 0/900 [00:00<?, ?it/s]

(np.float64(72806.49541548973),
 [np.float64(72321.66558357302),
  np.float64(73480.13829648319),
  np.float64(72617.68236641302)])

# LGB3

In [197]:
hparams = {
    'model_params': {
        'num_leaves': 7, 'n_estimators': 150, 'learning_rate': 0.05, 'colsample_bytree': 0.25, 'subsample': 0.5, 'metric': 'RMSE',
        'subsample_freq': 1
    }, 
    'X_num' : ['model_year', 'milage', 'clean_title', 'HP', 'displacement', 'speed'],
    'X_cat' : ['brand2', 'model2', 'fuel_type', 'ext_col', 'int_col', 'accident', 'engine_type', 'fuel_type_engine', 'method'],
    'cat': {'handle_unknown': 'use_encoded_value', 'unknown_value': -1, 'encoded_missing_value': -1},
    'validation_fraction': 0.0
}
result_lgb = sc.cv_result('lgb3', df_train, kf, hparams, config, lgb_adapter, rerun = 0)
np.mean(result_lgb['valid_scores']), result_lgb['valid_scores']

Fold:   0%|          | 0/3 [00:00<?, ?it/s]

Round:   0%|          | 0/150 [00:00<?, ?it/s]

Round:   0%|          | 0/150 [00:00<?, ?it/s]

Round:   0%|          | 0/150 [00:00<?, ?it/s]

(np.float64(73081.49055067466),
 [np.float64(72629.40406856804),
  np.float64(73753.79227921403),
  np.float64(72861.27530424189)])

In [189]:
result_lgb['model_result'][0]['valid_result'].idxmin()

metric  set     
rmse    training    149
        valid_1     118
dtype: int64

# CB4

In [209]:
hparams = {
    'model_params': {'max_depth': 3, 'n_estimators': 1000, 'learning_rate': 0.03, 'colsample_bylevel': 0.5, 'subsample': 0.5}, 
    'X_num' : ['model_year', 'milage', 'clean_title', 'HP', 'displacement', 'speed'],
    'X_cat' : ['brand2', 'model2', 'fuel_type', 'method', 'ext_col', 'int_col', 'accident', 'engine_type', 'fuel_type_engine'],
    'cat': {'handle_unknown': 'use_encoded_value', 'unknown_value': -1, 'encoded_missing_value': -1},
    'validation_fraction': 0.0
}
result_cb = sc.cv_result('cb4', df_train, kf, hparams, config, cb_adapter, rerun = 0)
np.mean(result_cb['valid_scores']), result_cb['valid_scores']

Fold:   0%|          | 0/3 [00:00<?, ?it/s]

Round:   0%|          | 0/1000 [00:00<?, ?it/s]

Round:   0%|          | 0/1000 [00:00<?, ?it/s]

Round:   0%|          | 0/1000 [00:00<?, ?it/s]

(np.float64(72832.85223388679),
 [np.float64(72355.21712498991),
  np.float64(73502.74606871625),
  np.float64(72640.59350795418)])

In [210]:
result_cb['model_result'][0]['valid_result'].idxmin()

metric  set  
RMSE    learn    999
dtype: int64

# NN1

In [5]:
nn_adapter = sgnn.NNAdapter(model=sgnn.NNRegressor)

In [20]:
def nn_cat_param(df, name, size):
    return name, len(df[name].cat.categories), size

X_nn_emb = [
    ('brand2', 3), ('model2', 3), ('fuel_type', 2), ('method', 2), ('ext_col', 3), ('int_col', 3), ('engine_type', 2), ('fuel_type_engine', 3)
]
X_nn_cat = [i for i,_ in X_nn_emb]

nn_params = {
    'embedding':[(1, c, es, 0, 0) for _, c, es in [nn_cat_param(df_train, v, n) for v, n in X_nn_emb]], 
    'config':  [
        {'unit': 16, 'activation': 'relu', 'batch_norm': False},
        {'unit': 16, 'activation': 'relu', 'batch_norm': False},
        {'unit': 8, 'activation': 'relu', 'batch_norm': False},
    ]
}

hparams = {
    'model_params': {
        'model_params': nn_params,
        'epochs': 20,
        'optimizer': ('Adam', {'learning_rate': 0.01}),
        'batch_size': 2048, 'shuffle_size': 102400,
        'early_stopping': None, 'reduce_lr_on_plateau': None, 'lr_scheduler': None,
    }, 
    'X_cat': X_nn_cat, 'cat': {'handle_unknown': 'use_encoded_value', 'unknown_value': -1, 'encoded_missing_value': -1},
    'X_std': ['model_year'], 'X_std': ['milage', 'displacement2', 'HP2'],
    'X_ohe': ['accident'], 
    'ohe': {'drop': 'first', 'sparse_output': False},
    'validation_fraction': 0.0,
    'metrics': 'RMSE'
}
result_nn = sc.cv_result('nn1', df_train, kf, hparams, config, nn_adapter, rerun = 0)
np.mean(result_nn['valid_scores']), result_nn['valid_scores']

Fold:   0%|          | 0/3 [00:00<?, ?it/s]

Epoch:   0%|          | 0/20 [00:00<?, ?it/s]

Step:   0%|          | 0/62 [00:00<?, ?it/s]

Epoch:   0%|          | 0/20 [00:00<?, ?it/s]

Step:   0%|          | 0/62 [00:00<?, ?it/s]

Epoch:   0%|          | 0/20 [00:00<?, ?it/s]

Step:   0%|          | 0/62 [00:00<?, ?it/s]

(np.float64(73104.99899435883),
 [np.float64(72700.8024253999),
  np.float64(73730.15858543642),
  np.float64(72884.03597224019)])

# Ensemble1

In [211]:
df_prds = sc.read_prds(['lgb1', 'lgb2', 'cb1', 'cb2', 'xgb1', 'xgb2', 'cb3', 'lgb3', 'cb4'], index = df_train.index)
df_prds.apply(lambda x: root_mean_squared_error(df_train[target], x))

lgb1    73127.571780
lgb2    73105.272968
cb1     72828.405655
cb2     72792.589378
xgb1    73103.675103
xgb2    73182.408307
cb3     72808.151342
lgb3    73083.095624
cb4     72834.483231
dtype: float64

In [222]:
root_mean_squared_error(
    df_train[target], df_prds.dot([0.1, 0.1, 0.15, 0.45, 0.10, 0.0, 0.1, 0.0, 0.0])
)

np.float64(72750.85299138822)

In [5]:
for i in ['lgb1', 'lgb2', 'cb1', 'cb2', 'xgb1', 'xgb2', 'cb3', 'lgb3', 'cb4']:
    sc.train_cv(i, df_train, config)

Round:   0%|          | 0/100 [00:00<?, ?it/s]

Round:   0%|          | 0/100 [00:00<?, ?it/s]

Round:   0%|          | 0/350 [00:00<?, ?it/s]

Round:   0%|          | 0/850 [00:00<?, ?it/s]

Round:   0%|          | 0/550 [00:00<?, ?it/s]

Round:   0%|          | 0/550 [00:00<?, ?it/s]

Round:   0%|          | 0/900 [00:00<?, ?it/s]

Round:   0%|          | 0/150 [00:00<?, ?it/s]

Round:   0%|          | 0/1000 [00:00<?, ?it/s]

In [8]:
df_test_prds = pd.concat([
    sc.get_predictor_cv('lgb1', config)(df_test).rename(i)
    for i in ['lgb1', 'lgb2', 'cb1', 'cb2', 'xgb1', 'xgb2', 'cb3', 'lgb3', 'cb4']
], axis=1)

In [10]:
df_test_prds.dot([0.1, 0.1, 0.15, 0.45, 0.10, 0.0, 0.1, 0.0, 0.0]).rename(target).to_csv('result/submission1.csv')

In [11]:
!head result/submission1.csv

id,price
188533,18604.219874272057
188534,71849.47892200263
188535,60042.44465549229
188536,30567.95639662654
188537,29740.041672416664
188538,18759.48439074103
188539,14450.789839722302
188540,43246.37255721986
188541,72309.4818039167


In [ ]:
# !kaggle competitions submit -c playground-series-s4e9 -f result/submission1.csv -m "1"

# Ensemble2

In [21]:
df_prds = sc.read_prds(['lgb1', 'lgb2', 'cb1', 'cb2', 'xgb1', 'xgb2', 'cb3', 'lgb3', 'cb4', 'nn1'], index = df_train.index)
df_prds.apply(lambda x: root_mean_squared_error(df_train[target], x))

lgb1    73127.571780
lgb2    73105.272968
cb1     72828.405655
cb2     72792.589378
xgb1    73103.675103
xgb2    73182.408307
cb3     72808.151342
lgb3    73083.095624
cb4     72834.483231
nn1     73106.371626
dtype: float64

In [22]:
root_mean_squared_error(
    df_train[target], df_prds.dot([0.1, 0.1, 0.10, 0.4, 0.10, 0.0, 0.1, 0.0, 0.0, 0.1])
)

np.float64(72738.71255275169)

In [23]:
for i in ['lgb1', 'lgb2', 'cb1', 'cb2', 'xgb1', 'xgb2', 'cb3', 'lgb3', 'cb4', 'nn1']:
    sc.train_cv(i, df_train, config)

Epoch:   0%|          | 0/20 [00:00<?, ?it/s]

Step:   0%|          | 0/93 [00:00<?, ?it/s]

In [26]:
df_test_prds = pd.concat([
    sc.get_predictor_cv(i, config)(df_test).rename(i)
    for i in ['lgb1', 'lgb2', 'cb1', 'cb2', 'xgb1', 'xgb2', 'cb3', 'lgb3', 'cb4', 'nn1']
], axis=1)

In [28]:
df_test_prds.dot([0.1, 0.1, 0.1, 0.4, 0.10, 0.0, 0.1, 0.0, 0.0, 0.1]).rename(target).to_csv('result/submission2.csv')

In [29]:
!head result/submission2.csv

id,price
188533,17589.7115989793
188534,80134.52699272221
188535,57413.749680587396
188536,27594.13238763532
188537,28797.925494766965
188538,18700.26500615405
188539,12933.05167696676
188540,43900.219372453386
188541,82697.61765316878


- 현재 분석 루틴의 유효성으 보기 위해서 만들어 봤습니다. 짜잘한 버그를 수정을 했고, 이제 시작한 competition을 참가하기 위해 이만 마칩니다.